In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers, optimizers
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

# Параметри директорій з даними
train_dir = r'C:\Users\tatya\Downloads\cat@dog\train'
test_dir = r'C:\Users\tatya\Downloads\cat@dog\test'

# Загружаємо передньо навчену модель VGG16 без повноівязаних шарів
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Заморожуємо згорткову базу
conv_base.trainable = False

# Створюємо генератори зображень з аугментацією для тренувальної та валідаційної вибірок
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

# Створюємо модель, додаючи повноівязані шари на верх передньо навченої VGG16
model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Компілюємо модель з правильним налаштуванням оптимізатора
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),  
              metrics=['acc'])

# Налаштовуємо TensorBoard для візуалізації
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Навчаємо модель
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50,
    callbacks=[tensorboard_callback]
)

# Візуалізація результатів в TensorBoard:
# %load_ext tensorboard
# %tensorboard --logdir logs


Found 25000 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.
Epoch 1/5


c:\users\tatya\appdata\local\programs\python\python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - acc: 0.9949 - loss: 0.0616 - val_acc: 1.0000 - val_loss: 4.5565e-04
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 254s 3s/step - acc: 1.0000 - loss: 2.9530e-04 - val_acc: 1.0000 - val_loss: 7.8810e-05
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - acc: 1.0000 - loss: 6.4923e-05 - val_acc: 1.0000 - val_loss: 3.7627e-05
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - acc: 1.0000 - loss: 3.5350e-05 - val_acc: 1.0000 - val_loss: 2.6198e-05
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - acc: 1.0000 - loss: 2.5185e-05 - val_acc: 1.0000 - val_loss: 1.8300e-05
